# Creating a clustered tsp dataset

In [2]:
"""Import Statements"""
from concorde.tsp import TSPSolver
import numpy as np
from sklearn import datasets
from IPython.display import clear_output

from src import fileops as fo

In [4]:
def get_random_cluster():
    """
        Create one random clustered instance with limits (0, 10000)
        |V| = 300 - 600
        |Centers| = 3-6
    """
    # unbalanced tuple unpacking is intended, we don't need Number of Clusters
    # pylint: disable=W0632
    blobs, _ = datasets.make_blobs(
        n_samples=np.random.randint(low=300,high=600),
        center_box=(1500,8500),
        cluster_std=400,
        centers = np.random.randint(low=3, high=7))

    blobs = blobs.astype(int)
    for point in blobs:
        if point[0] < 0:
            point[0] = 0
        if point[0] > 10000:
            point[0] = 10000
        if point[1] < 0:
            point[1] = 0
        if point[1]> 10000:
            point[1] = 10000
    return blobs

In [5]:
def solve_tsp(coordinates):
    """
        Call the Concorde Solver for given coordinates data
    """
    solver = TSPSolver.from_data(
        coordinates.T[0],
        coordinates.T[1],
        norm="EUC_2D"
    )
    tour_data = solver.solve(time_bound = 60.0,verbose = False, random_seed = 42)
    return tour_data.tour, tour_data.optimal_value

## Create the dataset

In [3]:
FOLDER_BASE_PATH = 'tspsetX/ctsp_'
INSTANCES = 2000

In [ ]:
for i in range(INSTANCES):
    print(f'COMPUTING INSTANCE {i} OF {INSTANCES}')
    instance = get_random_cluster()
    tour, opt = solve_tsp(instance)
    result = {
        "comment": f'randomly generated clustered problem with {len(instance)} cities',
        "dimension": len(instance),
        "display_data_type": "COORD_DISPLAY",
        "edge_weight_type": "EUC_2D",
        "name": i,
        "node_coordinates": instance.tolist(),
        "tour": tour.tolist(),
        "tourlength": int(opt),
        "type": "tsp"
    }
    fo.save_as_json(result, f'{FOLDER_BASE_PATH}{i}_sol.json')
    clear_output()